In [2]:
import os
import time

import duckdb
import numpy as np
import polars as pl
import psutil


# Função para medir uso de memória
def memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)  # Convertendo para MB


# Simulando um DataFrame com 1 milhão de linhas
n = 1_000_000
data = {
    "idade": np.random.randint(1000, 5000, size=n),
    "other_column1": np.random.random(size=n),
    "other_column2": np.random.random(size=n),
    "other_column3": np.random.random(size=n),
    "other_column4": np.random.random(size=n),
    "other_column5": np.random.random(size=n),
    "other_column6": np.random.random(size=n),
    "other_column7": np.random.random(size=n),
    "other_column8": np.random.random(size=n),
    "other_column9": np.random.random(size=n),
    "other_column10": np.random.random(size=n),
    "other_column11": np.random.random(size=n),
    "other_column12": np.random.random(size=n),
    "other_column13": np.random.random(size=n),
    "other_column14": np.random.random(size=n),
    "other_column15": np.random.random(size=n),
    "other_column16": np.random.random(size=n),
    "other_column17": np.random.random(size=n),
    "other_column18": np.random.random(size=n),
    "other_column19": np.random.random(size=n),
    "other_column20": np.random.random(size=n),
    "other_column21": np.random.random(size=n),
    "other_column22": np.random.random(size=n),
}


def age_decoder(idade, unidade: str = "Y"):
    fator = {"Y": 1.0, "M": 12.0, "D": 365.0, "H": 365 * 24.0}
    idade = int(idade)
    if idade >= 4000:  # idade em anos
        idade_anos = idade - 4000
    elif idade >= 3000 and idade < 4000:  # idade em meses
        idade_anos = (idade - 3000) / 12.0
    elif idade >= 2000 and idade < 3000:  # idade em dias
        idade_anos = (idade - 2000) / 365.0
    elif idade >= 1000 and idade < 2000:  # idade em horas
        idade_anos = (idade - 1000) / (365 * 24.0)
    else:
        idade_anos = np.nan
    idade_dec = idade_anos * fator[unidade]
    return idade_dec


# DuckDB method
def transform_age_duckdb():
    # Criar uma conexão DuckDB em memória
    con = duckdb.connect(database=":memory:")

    # Criar uma tabela em DuckDB a partir do DataFrame Pandas
    con.execute("CREATE TABLE my_table AS SELECT * FROM df_pandas")

    # Medir uso de memória antes da execução
    memory_before = memory_usage()

    # Consulta SQL para aplicar a lógica do age_decoder
    start_duckdb = time.time()
    result_duckdb = con.execute("""
        SELECT idade,
            CASE 
                WHEN idade >= 4000 THEN (idade - 4000) * 1.0
                WHEN idade >= 3000 AND idade < 4000 THEN (idade - 3000) / 12.0
                WHEN idade >= 2000 AND idade < 3000 THEN (idade - 2000) / 365.0
                WHEN idade >= 1000 AND idade < 2000 THEN (idade - 1000) / (365 * 24.0)
                ELSE NULL
            END AS idade_decodificada,
            other_column
        FROM my_table
    """).fetchdf()
    end_duckdb = time.time()

    # Medir uso de memória após a execução
    memory_after = memory_usage()

    return (end_duckdb - start_duckdb), (memory_after - memory_before)


# Polars method
def transform_age_polars():
    df_polars = pl.DataFrame(data)

    # Medir uso de memória antes da execução
    memory_before = memory_usage()

    # Aplicar a função age_decoder com Polars
    start_polars = time.time()
    df_polars = df_polars.with_columns(
        pl.col("idade")
        .apply(lambda idade: age_decoder(idade), return_dtype=pl.Float64)
        .alias("idade_decodificada")
    )
    end_polars = time.time()

    # Medir uso de memória após a execução
    memory_after = memory_usage()

    return (end_polars - start_polars), (memory_after - memory_before)


# Medindo o tempo de execução e memória para DuckDB e Polars
time_duckdb, mem_duckdb = transform_age_duckdb()
time_polars, mem_polars = transform_age_polars()

print(
    f"Tempo de execução DuckDB: {time_duckdb} segundos, Memória usada: {mem_duckdb} MB"
)
print(
    f"Tempo de execução Polars: {time_polars} segundos, Memória usada: {mem_polars} MB"
)

CatalogException: Catalog Error: Table with name df_pandas does not exist!
Did you mean "pg_database"?
LINE 1: CREATE TABLE my_table AS SELECT * FROM df_pandas
                                               ^